In [7]:
import numpy as np
import pandas as pd
import pymysql
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [3]:
connection = pymysql.connect("localhost", "root", "", "moviereviews")
ds = pd.read_sql_query("SELECT * from movies_movies", connection)

In [16]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(ds['title'])

In [17]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [18]:
results = {}

for idx, row in ds.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], ds['id'][i]) for i in similar_indices]
    results[row['id']] = similar_items[1:]
    
print('done!')


done!


In [11]:
def item(id):
    return (ds.loc[ds['id'] == id]['id']).tolist()[0]

def recommend(item_id, num):
    ids = []
    recs = results[item_id][:num]
    for rec in recs:
        value = (ds.loc[ds['id'] == rec[1]]['id']).tolist()[0]
        ids.append(value)
    df = ds[ds['id'].isin(ids)]
    return df
        


In [19]:
%%time
recommend(item_id=1, num=5)

Wall time: 13 ms


,id,title,genres
2131,2491,"Story of Us, The (1999)",Comedy|Drama
2259,2619,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy
3499,3859,"Toy, The (1982)",Comedy
3993,4353,Toy Soldiers (1991),Action|Drama
7259,7619,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX
